In [2]:
import copy
import csv
import os
import re
import warnings
import random as python_random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


In [3]:
admissions = pd.read_csv("hosp/admissions.csv")
patients = pd.read_csv("hosp/patients.csv")
diagnoses = pd.read_csv("hosp/diagnoses_icd.csv")
icustays = pd.read_csv("icu/icustays.csv")
chartevents = pd.read_csv("icu/chartevents.csv", chunksize=250000)  # Adjusted chunk size
labevents = pd.read_csv("hosp/labevents.csv")
discharge = pd.read_csv("discharge.csv")


In [ ]:
data = pd.read_csv(f"diagnoses_icd.csv")
for i in ['I5030', '42830', 'I5031', '42831', 'I5032', '42832', 'I5033', '42833']:
    count = data[data.icd_code == i]#.subject_id.count()#nunique()
    #['subject_id','hadm_id']#.nunique()#.count()
    #count.to_csv("test.csv")
    if i == "I5030":
        new_df = pd.DataFrame(count)
    else:
        new_df = pd.concat([new_df, count], ignore_index=True)
    print(i)


In [4]:
#discharge = pd.read_csv("discharge.csv")
LVEF = discharge[discharge['text'].str.contains(r'LVEF')]

In [5]:
#LVEF_per = LVEF_new['text'].str.findall(r'LVEF.+?(?=%)')
#print(LVEF_per)
#LVEF_per = LVEF_new.loc[LVEF_new['text'].str.contains(r'LVEF.+?%', na=False)]
#print(LVEF_per)
LVEF_per = LVEF.copy()
LVEF_per['text'] = LVEF_per['text'].str.extract(r'(LVEF.+?%)')
print(LVEF_per)


               note_id  subject_id   hadm_id note_type  note_seq  \
9       10000764-DS-11    10000764  27897940        DS        11   
10      10000826-DS-17    10000826  20032235        DS        17   
18      10000980-DS-20    10000980  29654838        DS        20   
19      10000980-DS-21    10000980  26913865        DS        21   
24      10000980-DS-26    10000980  20897796        DS        26   
...                ...         ...       ...       ...       ...   
331782  19999784-DS-22    19999784  29889147        DS        22   
331783  19999784-DS-23    19999784  29956342        DS        23   
331784  19999784-DS-24    19999784  24755486        DS        24   
331790  19999840-DS-20    19999840  26071774        DS        20   
331792   19999987-DS-2    19999987  23865745        DS         2   

                  charttime            storetime         text  
9       2132-10-19 00:00:00  2132-10-19 18:50:00          NaN  
10      2146-12-12 00:00:00  2146-12-12 18:22:00    LVE

In [6]:
LVEF_gt = LVEF_per.copy()
LVEF_gt['text'] = LVEF_gt['text'].str.extract(r'(>.*[0-9]*)')
LVEF_gt = LVEF_gt.dropna(subset=['text'])
#print(LVEF_gt)
#print(LVEF_gt['text'].unique())
LVEF_gt['text'] = LVEF_gt['text'].str.extract(r'([0-9]+)')
#print(LVEF_gt['text'].unique())
gt_values = LVEF_gt.text.value_counts()
print(gt_values)

text
55    20087
60      404
75      225
65      210
70      130
50       68
40       10
45        5
47        2
43        1
20        1
64        1
30        1
32        1
80        1
71        1
25        1
58        1
Name: count, dtype: int64


In [7]:
LVEF_lt = LVEF_per.copy()
LVEF_lt['text'] = LVEF_lt['text'].str.extract(r'(<[<]*[ ]*[0-9]*)')
LVEF_lt = LVEF_lt.dropna(subset=['text'])
#print(LVEF_lt)
print(LVEF_lt['text'].unique())
LVEF_lt['text'] = LVEF_lt['text'].str.extract(r'([0-9]+)')
print(LVEF_lt['text'].unique())
lt_values = LVEF_lt.text.value_counts()
print(lt_values)

['<20' '< 20' '<40' '<35' '<30' '< 10' '<<20' '<' '< 30' '<10' '< 15'
 '<25' '< 40' '<15' '<<10' '< 25' '< 45']
['20' '40' '35' '30' '10' nan '15' '25' '45']
text
20    144
30     25
40     12
10      8
25      8
15      7
35      4
45      1
Name: count, dtype: int64


In [8]:
LVEF_et = LVEF_per.copy()
LVEF_et['text'] = LVEF_et['text'].str.extract(r'(=.*[0-9])')
LVEF_et = LVEF_et.dropna(subset=['text'])
#print(LVEF_et)
print(LVEF_et['text'].unique())
LVEF_et['text'] = LVEF_et['text'].str.extract(r'([0-9]+)')
print(LVEF_et['text'].unique())
et_values = LVEF_et.text.value_counts()
print(et_values)

['= ? 35-40' '= 25' '= 40-45' '= 64' '= 35-40' '= 65' '= 45-50' '= 59'
 '= 20' '= 68' '= 40' '= 35' '= 61' '= 79' '= 55' '= 60' '= 26' '= 15'
 '=45' '= 46' '= 40 - 45' '= 50-55' '= 72' '= 63' '= 41' '= 50' '= 21'
 '= 30' '= 45' '= 70' '=75' '= 13' '=25' '=64' '= 58' '= 36' '= 47' '= 53'
 '=40' '= 71' '= 52' '=55' '= 69' '= 57' '= 17' '= 56' '= 29' '=63' '=42'
 '= 62' '= 44' '= 55-60' '= 30 - 35' '=50' '= ? 40' '=59' '=55-60' '= 43'
 '=40-45' '= 27' '= 10 - 15' '= 67' '= <20' '= 20 - 25' '=70' '= 49'
 '= 33' '= 48' '=56' '= 15 - 20' '= 38' '=65' '=50-60' '= 45 - 50' '= 54'
 '=60' '= 60-65' '= 22' '= 25 - 30' '=30' '= 40-50' '= 37' '= 23'
 '= 15*-20' '= 42' '= >55' '= 24' '= 10' '=20' '= 66' '= 75' '= 34' '= 73'
 '= 3040' '= 14' '=58' '= 32' '=35' '= 19' '=30 to 40' '= ? 30' '= 74'
 '= 60-70' '= ?45-50' '= 77' '= ?40' '= 35- 40' '= 31' '= ?45' '= ~50'
 '= 35-45' '= < 10' '= 51' '= 50 - 55' '=45-50' '= 65-70' '=53' '= >60'
 '= 25 -30' '= 28' '= 45 to 50' '= 80' '= 78' '= 35 to 40' '= 39'


In [9]:
#print(gt_values, lt_values, et_values)
under_40 = sum([
    (LVEF_et['text'].astype(float) <= 40).sum(),
    (LVEF_gt['text'].astype(float) <= 40).sum(),
    (LVEF_lt['text'].astype(float) <= 40).sum()
])

between_40_50 = sum([
    ((LVEF_et['text'].astype(float) > 40) & (LVEF_et['text'].astype(float) <= 50)).sum(),
    ((LVEF_gt['text'].astype(float) > 40) & (LVEF_gt['text'].astype(float) <= 50)).sum(),
    ((LVEF_lt['text'].astype(float) > 40) & (LVEF_lt['text'].astype(float) <= 50)).sum()
])

over_50 = sum([
    (LVEF_et['text'].astype(float) > 50).sum(),
    (LVEF_gt['text'].astype(float) > 50).sum(),
    (LVEF_lt['text'].astype(float) > 50).sum()
])

print(f"Count under 40: {under_40}")
print(f"Count between 40 and 50: {between_40_50}")
print(f"Count over 50: {over_50}")

Count under 40: 4414
Count between 40 and 50: 2218
Count over 50: 24038


In [10]:
LVEF_all = pd.concat([pd.concat([LVEF_et, LVEF_gt], ignore_index=True),LVEF_lt], ignore_index=True)
print(LVEF_all)

              note_id  subject_id   hadm_id note_type  note_seq  \
0      10002495-DS-13    10002495  24982426        DS        13   
1      10004235-DS-21    10004235  24181354        DS        21   
2      10004401-DS-27    10004401  29988601        DS        27   
3      10004720-DS-11    10004720  22081550        DS        11   
4      10005817-DS-11    10005817  20626031        DS        11   
...               ...         ...       ...       ...       ...   
30683   19810932-DS-3    19810932  29764035        DS         3   
30684  19818243-DS-19    19818243  25296595        DS        19   
30685   19968075-DS-2    19968075  28592225        DS         2   
30686  19993336-DS-11    19993336  24615303        DS        11   
30687  19993336-DS-12    19993336  23077223        DS        12   

                 charttime            storetime text  
0      2141-05-29 00:00:00  2141-05-30 02:29:00   35  
1      2196-03-04 00:00:00  2196-03-06 10:57:00   25  
2      2144-02-06 00:00:00  21

In [11]:
LVEF_all = LVEF_all.drop(['note_type', 'note_seq'], axis=1)
print(LVEF_all)

              note_id  subject_id   hadm_id            charttime  \
0      10002495-DS-13    10002495  24982426  2141-05-29 00:00:00   
1      10004235-DS-21    10004235  24181354  2196-03-04 00:00:00   
2      10004401-DS-27    10004401  29988601  2144-02-06 00:00:00   
3      10004720-DS-11    10004720  22081550  2186-11-17 00:00:00   
4      10005817-DS-11    10005817  20626031  2132-12-20 00:00:00   
...               ...         ...       ...                  ...   
30683   19810932-DS-3    19810932  29764035  2148-12-18 00:00:00   
30684  19818243-DS-19    19818243  25296595  2144-03-07 00:00:00   
30685   19968075-DS-2    19968075  28592225  2153-04-23 00:00:00   
30686  19993336-DS-11    19993336  24615303  2171-09-12 00:00:00   
30687  19993336-DS-12    19993336  23077223  2171-10-04 00:00:00   

                 storetime text  
0      2141-05-30 02:29:00   35  
1      2196-03-06 10:57:00   25  
2      2144-02-06 10:07:00   40  
3      2186-11-18 20:37:00   64  
4      2132-1

In [12]:
LVEF_all = LVEF_all.rename(columns={"text": "LVEF"})
print(LVEF_all)


              note_id  subject_id   hadm_id            charttime  \
0      10002495-DS-13    10002495  24982426  2141-05-29 00:00:00   
1      10004235-DS-21    10004235  24181354  2196-03-04 00:00:00   
2      10004401-DS-27    10004401  29988601  2144-02-06 00:00:00   
3      10004720-DS-11    10004720  22081550  2186-11-17 00:00:00   
4      10005817-DS-11    10005817  20626031  2132-12-20 00:00:00   
...               ...         ...       ...                  ...   
30683   19810932-DS-3    19810932  29764035  2148-12-18 00:00:00   
30684  19818243-DS-19    19818243  25296595  2144-03-07 00:00:00   
30685   19968075-DS-2    19968075  28592225  2153-04-23 00:00:00   
30686  19993336-DS-11    19993336  24615303  2171-09-12 00:00:00   
30687  19993336-DS-12    19993336  23077223  2171-10-04 00:00:00   

                 storetime LVEF  
0      2141-05-30 02:29:00   35  
1      2196-03-06 10:57:00   25  
2      2144-02-06 10:07:00   40  
3      2186-11-18 20:37:00   64  
4      2132-1

In [13]:
# Filter diagnoses for relevant ICD codes
heart_failure_patients = diagnoses[diagnoses["icd_code"].isin(['I5030', '42830', 'I5031', '42831', 'I5032', '42832', 'I5033', '42833'])]

# Merge with LVEF
LVEF_patients = heart_failure_patients.merge(LVEF_all, on=["subject_id", "hadm_id"], how="inner")
# Merge with ICU stays
hf_icu = LVEF_patients.merge(icustays, on=["subject_id", "hadm_id"], how="inner")

# Extract unique ICU stay IDs
icu_ids = hf_icu["stay_id"].unique()
print(hf_icu, len(icu_ids))

      subject_id   hadm_id  seq_num icd_code  icd_version         note_id  \
0       10002013  23581541        2    42832            9   10002013-DS-7   
1       10010471  29842315        5    I5033           10  10010471-DS-22   
2       10011938  29339128        3    42831            9  10011938-DS-16   
3       10013643  22009484        2    42833            9  10013643-DS-21   
4       10014354  27487226        4    I5033           10  10014354-DS-18   
...          ...       ...      ...      ...          ...             ...   
2630    19997367  21508795       16    42832            9  19997367-DS-20   
2631    19997448  23560173        3    I5030           10  19997448-DS-13   
2632    19997448  23560173        3    I5030           10  19997448-DS-13   
2633    19998591  24349193        5    42832            9   19998591-DS-8   
2634    19998591  24349193        5    42832            9   19998591-DS-8   

                charttime            storetime LVEF   stay_id  \
0     2160

In [ ]:
# Define ITEMIDs for relevant vitals (e.g., Heart Rate, BP, SpO2)
# Define ITEMIDs for vitals and labs
itemid_map = {
    "Heart Rate": 220045,
    "Systolic BP": 220179,
    "Diastolic BP": 220180,
    "SpO2": 220277,
    "Temperature": 223761,
    "BMI": 226512,
    "Bicarbonate": 227443,
    "Creatinine": 220615,
    "Hemoglobin": 220228,
    "INR(PT)": 220562,
    "Platelet Count": 227457,
    "Potassium": 220640,
    "WBC Count": 220546,
    "Sodium": 220645,
    "NT-proBNP": 227444,
    "Troponin T": 227429
}

# Check unique values for 'stay_id' and 'itemid' in a sample chunk
vitals_list = []
print("Unique ICU IDs:", icu_ids)
print(f"Number of unique ICU IDs: {len(icu_ids)}")

# Load relevant data
chunk_size = 10000  # Adjust the chunk size based on your available memory
vitals_list = []
for chunk in pd.read_csv("icu/chartevents.csv", chunksize=chunk_size):
    #chunk_filtered = chunk[(chunk["stay_id"].isin(icu_ids)) & (chunk["itemid"].isin(itemid_map.values()))]
    #vitals_list.append(chunk_filtered)
    #print(vitals_list)
    print(chunk)

# Concatenate all filtered chunks
vitals = pd.concat(vitals_list, ignore_index=True)
print(vitals)  # Check if the vitals dataframe is populated


In [82]:
print(vitals)

        subject_id   hadm_id   stay_id  caregiver_id           charttime  \
235482    12106780  25661012  37081503       17994.0 2110-02-04 10:31:00   
235619    12106780  25661012  37081503       96640.0 2110-02-04 12:19:00   
235618    12106780  25661012  37081503       96640.0 2110-02-04 12:19:00   
235622    12106780  25661012  37081503       96640.0 2110-02-04 12:25:00   
235620    12106780  25661012  37081503       96640.0 2110-02-04 12:25:00   
...            ...       ...       ...           ...                 ...   
465519    14288592  28211733  31011820       14454.0 2209-03-24 19:02:00   
465521    14288592  28211733  31011820       14454.0 2209-03-24 20:00:00   
465520    14288592  28211733  31011820       14454.0 2209-03-24 20:00:00   
465523    14288592  28211733  31011820       14454.0 2209-03-24 20:02:00   
465522    14288592  28211733  31011820       14454.0 2209-03-24 20:02:00   

                  storetime  itemid value  valuenum valueuom  warning  
235482  2110-02

In [83]:
# Convert charttime to datetime
vitals["charttime"] = pd.to_datetime(vitals["charttime"])
LVEF_all["charttime"] = pd.to_datetime(LVEF_all["charttime"])

# Sort by charttime (required for merge_asof)
vitals = vitals.sort_values("charttime")
LVEF_all = LVEF_all.sort_values("charttime")

print(LVEF_all)

# Merge on the closest (previous) timestamp
LVEF_vitals_merged = pd.merge_asof(
    vitals,  # Larger dataframe (vitals)
    LVEF_all,  # LVEF dataframe (smaller)
    on="charttime",
    by=["subject_id", "hadm_id"],  # Match on subject & admission ID
    direction="backward"  # Use "nearest" or "forward" if needed
)

LVEF_vitals_merged["LVEF"] = LVEF_vitals_merged.groupby("subject_id")["LVEF"].ffill()

print(LVEF_vitals_merged)


              note_id  subject_id   hadm_id  charttime            storetime  \
2088   12257372-DS-21    12257372  23964251 2110-01-19  2110-01-19 11:53:00   
16477  13401124-DS-17    13401124  27101384 2110-02-10  2110-02-10 20:23:00   
9680   10187951-DS-16    10187951  28001767 2110-02-11  2110-02-11 12:17:00   
13785  12106780-DS-20    12106780  25661012 2110-02-12  2110-02-12 13:31:00   
24859  17408195-DS-17    17408195  24443323 2110-02-15  2110-02-16 16:22:00   
...               ...         ...       ...        ...                  ...   
2486   12648465-DS-42    12648465  22702738 2209-09-03  2209-09-03 22:14:00   
20452  15273135-DS-16    15273135  22681403 2210-01-16  2210-01-16 16:31:00   
9815   10252385-DS-21    10252385  21039270 2210-01-18  2210-01-20 20:09:00   
9816   10252385-DS-22    10252385  27903520 2210-02-01  2210-02-01 21:13:00   
15940   13184296-DS-3    13184296  28785633 2211-05-07  2211-05-09 09:40:00   

      LVEF  
2088    45  
16477   55  
9680    55  

In [22]:
LVEF_vitals = vitals.merge(LVEF_all, on=["subject_id", "hadm_id"], how="inner")
print(LVEF_vitals)

LVEF_vitals["charttime_x"] = pd.to_datetime(LVEF_vitals["charttime_x"])
LVEF_vitals["storetime_x"] = pd.to_datetime(LVEF_vitals["storetime_x"])

# Pivot table: each ITEMID becomes a separate column
LVEF_vitals_pivot = LVEF_vitals.pivot_table(index=["subject_id", "stay_id", "charttime_x"], columns="itemid", values="valuenum")

# Reset column names
LVEF_vitals_pivot.columns = [str(col) for col in LVEF_vitals_pivot.columns]

# Reset index
LVEF_vitals_pivot.reset_index(inplace=True)

# Resample to hourly data (forward fill missing values)
LVEF_vitals_resampled = (
    LVEF_vitals_pivot.groupby("stay_id", group_keys=False)
    .apply(lambda group: group.reset_index().set_index("charttime_x").resample("1H").mean().ffill())
    .reset_index()
)

LVEF_vitals_resampled.drop(["caregiver_id", "warning", "note_id"], axis=1, inplace=True, errors="ignore")

print(LVEF_vitals_resampled)

        subject_id   hadm_id   stay_id  caregiver_id         charttime_x  \
0         12106780  25661012  37081503       96640.0 2110-02-04 12:19:00   
1         12106780  25661012  37081503       96640.0 2110-02-04 12:19:00   
2         12106780  25661012  37081503       96640.0 2110-02-04 12:25:00   
3         12106780  25661012  37081503       96640.0 2110-02-04 12:25:00   
4         12106780  25661012  37081503       17994.0 2110-02-04 12:30:00   
...            ...       ...       ...           ...                 ...   
912548    14288592  28211733  31011820       14454.0 2209-03-24 19:02:00   
912549    14288592  28211733  31011820       14454.0 2209-03-24 20:00:00   
912550    14288592  28211733  31011820       14454.0 2209-03-24 20:00:00   
912551    14288592  28211733  31011820       14454.0 2209-03-24 20:02:00   
912552    14288592  28211733  31011820       14454.0 2209-03-24 20:02:00   

                storetime_x  itemid value  valuenum valueuom  warning  \
0       2110-0

In [14]:
#test = LVEF_all[LVEF_all.duplicated(['subject_id', 'hadm_id'], keep=False)]
#print(test)
LVEF_all = LVEF_all.drop(['charttime', 'note_id', 'storetime'], axis=1)
print(LVEF_all)

      subject_id   hadm_id text icd_code
0       10010471  29842315   25    I5033
1       10014354  29600294   40    I5032
2       10018328  23786647   61    I5031
3       10018684  26649049   65    I5033
4       10028767  29267351   50    42833
...          ...       ...  ...      ...
6025    15100088  28332172   10    42833
6026    16500918  23702198   40    42831
6027    16924085  21803138   30    42832
6028    17760190  26479723   30    I5033
6029    19769933  26578430   20    42832

[6030 rows x 4 columns]


In [16]:
drgcodes = pd.read_csv(f"drgcodes.csv")
#drg = diagnosis related group
print(drgcodes)
LVEF_drg = LVEF_all.merge(drgcodes.drop_duplicates(), left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])
print(LVEF_drg)

        subject_id   hadm_id drg_type  drg_code  \
0         10000032  22595853      APR       283   
1         10000032  22595853     HCFA       442   
2         10000032  22841357      APR       279   
3         10000032  22841357     HCFA       442   
4         10000032  25742920      APR       283   
...            ...       ...      ...       ...   
761851    19999840  21033226     HCFA       100   
761852    19999840  26071774      APR        45   
761853    19999840  26071774     HCFA        65   
761854    19999987  23865745      APR        44   
761855    19999987  23865745     HCFA        64   

                                              description  drg_severity  \
0                            OTHER DISORDERS OF THE LIVER           2.0   
1       DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...           NaN   
2       HEPATIC COMA AND OTHER MAJOR ACUTE LIVER DISOR...           3.0   
3       DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...           NaN   
4           

In [20]:
#emar? medication administration
#patients -> gender
#omr!
#admissions?
#labevents, d_labitems?

patients = pd.read_csv(f"patients.csv")
patients = patients.drop(['anchor_age', 'anchor_year', 'anchor_year_group', 'dod'], axis=1)
print(patients)
omr = pd.read_csv(f"omr.csv")
omr = omr.drop(omr[omr.result_name == "Height (Inches)"].index)
omr = omr.drop(omr[omr.result_name == "Weight (Lbs)"].index)
print(omr)

        subject_id gender  anchor_age  anchor_year anchor_year_group  \
0         10000032      F          52         2180       2014 - 2016   
1         10000048      F          23         2126       2008 - 2010   
2         10000058      F          33         2168       2020 - 2022   
3         10000068      F          19         2160       2008 - 2010   
4         10000084      M          72         2160       2017 - 2019   
...            ...    ...         ...          ...               ...   
364622    19999828      F          46         2147       2017 - 2019   
364623    19999829      F          28         2186       2008 - 2010   
364624    19999840      M          58         2164       2008 - 2010   
364625    19999914      F          49         2158       2017 - 2019   
364626    19999987      F          57         2145       2011 - 2013   

               dod  
0       2180-09-09  
1              NaN  
2              NaN  
3              NaN  
4       2161-02-13  
...      

In [21]:
LVEF_drg_pat = LVEF_drg.merge(patients.drop_duplicates(), left_on=['subject_id'], right_on=['subject_id'])
print(LVEF_drg_pat)
LVEF_drg_pat_omr = LVEF_drg_pat.merge(omr.drop_duplicates(), left_on=['subject_id'], right_on=['subject_id'])
print(LVEF_drg_pat_omr)

       subject_id   hadm_id text icd_code drg_type  drg_code  \
0        10010471  29842315   25    I5033      APR       190   
1        10010471  29842315   25    I5033     HCFA       283   
2        10014354  29600294   40    I5032      APR       144   
3        10014354  29600294   40    I5032     HCFA       202   
4        10014354  27487226   55    I5033      APR       139   
...           ...       ...  ...      ...      ...       ...   
11322    16924085  21803138   30    42832     HCFA       309   
11323    17760190  26479723   30    I5033      APR       710   
11324    17760190  26479723   30    I5033     HCFA       853   
11325    19769933  26578430   20    42832      APR       203   
11326    19769933  26578430   20    42832     HCFA       313   

                                             description  drg_severity  \
0                            ACUTE MYOCARDIAL INFARCTION           4.0   
1             ACUTE MYOCARDIAL INFARCTION, EXPIRED W MCC           NaN   
2      RE

In [22]:
admissions = pd.read_csv(f"admissions.csv")
admissions = admissions.drop(['admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime'], axis=1)
print(admissions)
LVEF_drg_pat_omr_adm = LVEF_drg_pat_omr.merge(admissions.drop_duplicates(), left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])
print(LVEF_drg_pat_omr_adm)

        subject_id   hadm_id            admittime            dischtime  \
0         10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00   
1         10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00   
2         10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00   
3         10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00   
4         10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00   
...            ...       ...                  ...                  ...   
546023    19999828  25744818  2149-01-08 16:44:00  2149-01-18 17:00:00   
546024    19999828  29734428  2147-07-18 16:23:00  2147-08-04 18:10:00   
546025    19999840  21033226  2164-09-10 13:47:00  2164-09-17 13:42:00   
546026    19999840  26071774  2164-07-25 00:27:00  2164-07-28 12:15:00   
546027    19999987  23865745  2145-11-02 21:38:00  2145-11-11 12:57:00   

                  deathtime  hospital_expire_flag  
0                       NaN                     0  
1      

In [ ]:
labevents = pd.read_csv(f"labevents.csv")
print(labevents)
LVEF_drg_pat_omr_adm_lab = LVEF_drg_pat_omr_adm.merge(labevents.drop_duplicates(), left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])
print(LVEF_drg_pat_omr_adm_lab)

          labevent_id  subject_id  hadm_id  specimen_id   itemid  \
0                   1    10000032      NaN      2704548  50931.0   
1                   2    10000032      NaN     36092842  51071.0   
2                   3    10000032      NaN     36092842  51074.0   
3                   4    10000032      NaN     36092842  51075.0   
4                   5    10000032      NaN     36092842  51079.0   
...               ...         ...      ...          ...      ...   
32694744     32835703    12094256      NaN     94032513  50882.0   
32694745     32835704    12094256      NaN     94032513  50893.0   
32694746     32835705    12094256      NaN     94032513  50902.0   
32694747     32835706    12094256      NaN     94032513  50912.0   
32694748     32835707    12094256      NaN       940325      NaN   

         order_provider_id            charttime            storetime value  \
0                   P69FQC  2180-03-23 11:51:00  2180-03-23 15:56:00   ___   
1                   P69FQC 

In [ ]:
#NEW CODE

In [28]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm  # Progress bar for large operations

# Load core tables
admissions = pd.read_csv("hosp/admissions.csv")
patients = pd.read_csv("hosp/patients.csv")
diagnoses = pd.read_csv("hosp/diagnoses_icd.csv")
icustays = pd.read_csv("icu/icustays.csv")
chartevents = pd.read_csv("icu/chartevents.csv", chunksize=250000)  # Adjusted chunk size
labevents = pd.read_csv("hosp/labevents.csv")
discharge = pd.read_csv("discharge.csv")



In [29]:
# Normalize ICD codes
diagnoses["icd_code"] = diagnoses["icd_code"].str.replace(".", "", regex=True)

# Define relevant ICD codes
hf_icd_codes = {"I5030", "42830", "I5031", "42831", "I5032", "42832", "I5033", "42833"}

# Filter diagnoses for heart failure
heart_failure_patients = diagnoses[diagnoses["icd_code"].isin(hf_icd_codes)]

# Function to extract LVEF values from text
def extract_lvef(text):
    match = re.search(r'(\d+)', text)
    return int(match.group(1)) if match else np.nan

# Process LVEF values
LVEF = discharge[discharge['text'].str.contains(r'LVEF', na=False)].copy()
LVEF["LVEF"] = LVEF["text"].apply(extract_lvef)
LVEF.dropna(subset=["LVEF"], inplace=True)
LVEF = LVEF.drop(columns=["note_type", "note_seq"], errors='ignore')

# Merge LVEF with heart failure patients
LVEF_patients = heart_failure_patients.merge(LVEF, on=["subject_id", "hadm_id"], how="inner")

# Merge with ICU stays
hf_icu = LVEF_patients.merge(icustays, on=["subject_id", "hadm_id"], how="inner")

# Extract ICU stay IDs
icu_ids = set(hf_icu["stay_id"].unique())

# Define ITEMIDs for vitals and labs
itemid_map = {
    "Heart Rate": 220045,
    "Systolic BP": 220179,
    "Diastolic BP": 220180,
    "SpO2": 220277,
    "Temperature": 223761,
    "BMI": 226512,
    "Bicarbonate": 227443,
    "Creatinine": 220615,
    "Hemoglobin": 220228,
    "INR(PT)": 220562,
    "Platelet Count": 227457,
    "Potassium": 220640,
    "WBC Count": 220546,
    "Sodium": 220645,
    "NT-proBNP": 227444,
    "Troponin T": 227429
}

# Process chartevents in chunks
vitals_list = []
for chunk in tqdm(chartevents, desc="Processing chartevents"):
    chunk_filtered = chunk[(chunk["stay_id"].isin(icu_ids)) & (chunk["itemid"].isin(itemid_map.values()))]
    vitals_list.append(chunk_filtered)

# Concatenate all vitals data
vitals = pd.concat(vitals_list, ignore_index=True)

# Convert timestamps
vitals["charttime"] = pd.to_datetime(vitals["charttime"])
LVEF["charttime"] = pd.to_datetime(LVEF["charttime"])



Processing chartevents: 1732it [09:23,  3.07it/s]


In [43]:

print(vitals, LVEF)
# Merge admissions for age and mortality
admissions["admission_year"] = pd.to_datetime(admissions["admittime"]).dt.year
patients["dod"] = pd.to_datetime(patients["dod"], errors="coerce")
admissions["admittime"] = pd.to_datetime(admissions["admittime"])
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"])
admissions = admissions.merge(
    patients[["subject_id", "anchor_age", "anchor_year", "dod"]],
    on="subject_id",
    how="left",
    suffixes=("", "_patient")  # Avoids _x and _y issues
)
admissions["dod"] = pd.to_datetime(admissions["dod"], errors="coerce")  # Convert to datetime
admissions["dod"] = admissions["dod"].fillna(pd.Timestamp.max)  # Replace NaN with future date
admissions["death_within_30_days"] = ((admissions["dischtime"] + pd.Timedelta(days=30)) >= admissions["dod"]).astype(int)
admissions["death_within_1_year"] = ((admissions["dischtime"] + pd.Timedelta(days=365)) >= admissions["dod"]).astype(int)
admissions["age_admission"] = admissions["anchor_age"] + (admissions["admission_year"] - admissions["anchor_year"])

# Merge vitals with LVEF (closest previous measurement)
vitals.sort_values("charttime", inplace=True)
LVEF.sort_values("charttime", inplace=True)
LVEF_vitals = pd.merge_asof(vitals, LVEF, on="charttime", by=["subject_id", "hadm_id"], direction="backward")

# Forward-fill missing LVEF values per patient
LVEF_vitals["LVEF"] = LVEF_vitals.groupby("subject_id")["LVEF"].ffill()

# Merge with admissions
LVEF_vitals = LVEF_vitals.merge(admissions, on=["subject_id", "hadm_id"], how="left")

# Pivot to create time-series structure
LVEF_vitals_pivot = LVEF_vitals.pivot_table(index=["subject_id", "stay_id", "charttime"], columns="itemid", values="valuenum")
LVEF_vitals_pivot.columns = [str(col) for col in LVEF_vitals_pivot.columns]
LVEF_vitals_pivot.reset_index(inplace=True)

# Resample to hourly intervals
LVEF_vitals_pivot["charttime"] = pd.to_datetime(LVEF_vitals_pivot["charttime"])

# Resample to hourly intervals (without resetting index yet)
LVEF_vitals_resampled = (
    LVEF_vitals_pivot
    .set_index("charttime")  # Ensure charttime is the index
    .groupby("stay_id", group_keys=False)
    .resample("1H")  # Resample to 1 hour intervals
    .mean()  # Compute the mean for each hour
    .ffill()  # Forward-fill missing values
)

# Optimize memory usage
LVEF_vitals_resampled = LVEF_vitals_resampled.astype({col: "float32" for col in LVEF_vitals_resampled.columns if col not in ["subject_id", "stay_id", "charttime"]})

print(LVEF_vitals_resampled.head())


Empty DataFrame
Columns: [subject_id, hadm_id, stay_id, caregiver_id, charttime, storetime, itemid, value, valuenum, valueuom, warning]
Index: []                note_id  subject_id   hadm_id  charttime            storetime  \
66243   12024697-DS-11    12024697  20302177 2110-01-15  2110-01-19 10:31:00   
268863  18106347-DS-15    18106347  24305596 2110-01-15  2110-01-15 11:25:00   
73953   12257372-DS-21    12257372  23964251 2110-01-19  2110-01-19 11:53:00   
291493  18780420-DS-10    18780420  28977824 2110-01-22  2110-01-22 15:30:00   
158399  14779071-DS-15    14779071  20963511 2110-01-22  2110-01-22 12:54:00   
...                ...         ...       ...        ...                  ...   
124701  13774741-DS-10    13774741  27575074 2210-07-23  2210-07-24 23:12:00   
184399  15566010-DS-10    15566010  29192759 2210-09-11  2210-09-12 15:03:00   
77247    12352097-DS-8    12352097  23434751 2210-10-04  2210-10-05 15:48:00   
105230   13184296-DS-3    13184296  28785633 2211-05-0